<a href="https://colab.research.google.com/github/Auggustos/PCO102-2022.1-/blob/main/PCO102_01_Melhores_regressores_para_analise_sentimento_music_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introdução do problema
Neste notebook será realizada a seleção dos melhores regressores para analise de sentimento obtidos através de letras musicais. com os resultados poderemos ver os valores de sentimentos positivos negativos ou neutros de cada musica. estes valores serão os tutores para os regressores.

## Principal atributo a ser trabalhado

lyrics: Letra completa da música</br>

## Importação das bibliotecas 

In [ ]:
!pip install vaderSentiment
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import os
from textblob import TextBlob
import gdown
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

from sklearn import preprocessing
from sklearn import utils
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor


## Metodologias


A metodologia a ser utilizada neste trabalho é baseada na metodologia de outros dois trabalhos

o primeiro é o [Metodologias para Análise de Sentimentos de Tweets
sobre o Mercado Financeiro](https://bdm.unb.br/bitstream/10483/29207/1/2019_MuriloCerqueiraMedeiros_tcc.pdf). A metodologia apresentada neste TCC é descrita da seguinte forma: primeiramente, os tweets são carregados de um arquivo em disco, e são pré-processados, gerando-se um espaço vetorial. Em seguida, ocorre a extração de tópicos e a redução de dimensionalidade em paralelo. O resultado da redução de dimensionalidade é utilizado para classificar os sentimentos, para agrupar os tweets e para análise visual, a fim de tornar a investigação de aspectos em comum nos dados mais objetiva.

o segundo é o [Estudo e avaliação de métodos de análise de sentimentos baseada em aspectos para textos opinativos em português](https://www.teses.usp.br/teses/disponiveis/59/59143/tde-31102018-174346/publico/mateuscorrigida.pdf). A metodologia utilizada nesta tese é dividida em duas etapas, na primeira é identificada as caracteristicas do produto avaliado mencionados no texto, na segunda o foco é identificar a polaridade (positiva ou negativa) relacionado a cada aspecto do produto.

A princípio, neste trabalho é coletada a base de dados, realizado o pré-processamento da mesma, realizada a mineração do sentimento utilizando a biblioteca Vader, gerado a média dos valores agrupados por gênero, e comparação dos sentimentos entre os gêneros.

## Desenvolvimento

### Gdown pra baixar a base de dados

Aqui é definido o caminho no drive onde esta localizado o arquivo com a base de dados

In [ ]:
url = "https://drive.google.com/uc?id=1UgsRtv23jTZ4p2fCfe9Ccl3ASVMTTLLM"
output = "dataset_musicas.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UgsRtv23jTZ4p2fCfe9Ccl3ASVMTTLLM
To: /content/dataset_musicas.csv
100%|██████████| 27.7M/27.7M [00:00<00:00, 106MB/s]


'dataset_musicas.csv'

### Carregamento da base de dados

In [ ]:
dataset = pd.read_csv("/content/dataset_musicas.csv")

In [ ]:
dataset.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


### Pre-processamento

#### Remoção das colunas que não serão utilizadas

In [ ]:
dataset.columns.size

31

In [ ]:
dataset.drop(columns=["release_date","len", 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age', 'artist_name'], inplace=True)

In [ ]:
dataset.head()

,Unnamed: 0,track_name,genre,lyrics
0,0,mohabbat bhi jhoothi,pop,hold time feel break feel untrue convince spea...
1,4,i believe,pop,believe drop rain fall grow believe darkest ni...
2,6,cry,pop,sweetheart send letter goodbye secret feel bet...
3,10,patricia,pop,kiss lips want stroll charm mambo chacha merin...
4,12,apopse eida oneiro,pop,till darling till matter know till dream live ...


#### Remoção das stopwords

Ao analisar o conteúdo de algumas letras pertecentes ao dataset, percebi que as stopwords já foram removidas, desta forma o texto ja está pronto para ser enviado para o analisador de sentimento

### Analise de sentimento

In [ ]:
analyzer = SentimentIntensityAnalyzer()
df = pd.DataFrame(columns=["nome","genero","letra","positivo","negativo","neutro","compound"])

for index, row in dataset.iterrows():
  analisado = analyzer.polarity_scores(row['lyrics'])
  neu = analisado['neu']
  pos = analisado['pos']
  neg = analisado['neg']
  compound = analisado['compound']
  df = df.append({
      'nome' : row["track_name"], 
      'genero' : row["genre"], 
      'letra' : row["lyrics"],
      'positivo' : pos, 
      'negativo' : neg, 
      'neutro' : neu,
      'compound' : compound, 
      }, ignore_index=True)

df.head()



In [ ]:
df.shape

(28372, 7)

#### Salvamento da base pré-processada com sentimento

In [ ]:
df.to_csv("letras_analise_sentimento.csv")

#### Carregamento da base pré-processada

In [ ]:
url = "https://drive.google.com/uc?id=1HHDgs5LireyXYAYbzkh5fifwi43-JwS2"
output = "letras_analise_sentimento.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1HHDgs5LireyXYAYbzkh5fifwi43-JwS2
To: /content/letras_analise_sentimento.csv
100%|██████████| 14.2M/14.2M [00:00<00:00, 70.0MB/s]


'letras_analise_sentimento.csv'

In [ ]:
df = pd.read_csv("letras_analise_sentimento.csv")

In [ ]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.head()

,nome,genero,letra,positivo,negativo,neutro,compound
0,mohabbat bhi jhoothi,pop,hold time feel break feel untrue convince spea...,0.371,0.211,0.418,0.9398
1,i believe,pop,believe drop rain fall grow believe darkest ni...,0.150,0.259,0.591,-0.7717
2,cry,pop,sweetheart send letter goodbye secret feel bet...,0.305,0.000,0.695,0.8481
3,patricia,pop,kiss lips want stroll charm mambo chacha merin...,0.533,0.038,0.429,0.9887
4,apopse eida oneiro,pop,till darling till matter know till dream live ...,0.415,0.128,0.458,0.9623


#### Separação da base em treino e teste

In [ ]:
train, test = train_test_split(df, train_size=0.8)

criação do vetorizador de palavras

In [ ]:
vect_word = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
                        stop_words= 'english',dtype=np.float32)

Criação dos vetores de ngramas

In [ ]:
# Word ngram vector
tr_vect = vect_word.fit_transform(train['letra'])
ts_vect = vect_word.transform(test['letra'])

Colunas alvo

In [ ]:
target_col = ["positivo","negativo","neutro","compound"]
y = train[target_col]
X = tr_vect

#### definição dos regressores

In [ ]:
ln = MultiOutputRegressor(LinearRegression())
dtr = MultiOutputRegressor(DecisionTreeRegressor())
mlpr = MultiOutputRegressor(MLPRegressor())

#### treinamento e score de treinamento de cada regressor

In [ ]:
#Treinamento arvore decisão
%time dtr.fit(X,y)

CPU times: user 1min 12s, sys: 21.8 ms, total: 1min 12s
Wall time: 1min 12s


MultiOutputRegressor(estimator=DecisionTreeRegressor())

In [ ]:
#score de treinamento
dtr.score(X,y)

0.9999556414353725

In [ ]:
#Treinamento regressão linear
%time ln.fit(X,y)

CPU times: user 682 ms, sys: 3.03 ms, total: 685 ms
Wall time: 693 ms


MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
#score de treinamento
ln.score(X,y)

0.7782661286102168

In [ ]:
#Treinamento mlp
%time mlpr.fit(X,y)

CPU times: user 1min 34s, sys: 1min 21s, total: 2min 56s
Wall time: 1min 31s


MultiOutputRegressor(estimator=MLPRegressor())

In [ ]:
#score de treinamento
mlpr.score(X,y)

0.9906137667712863